### Figure 4 
- Sliding template during sleep deprivation
- Correlation across time window

### load data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from subjects import stat_kw
import plotters
import pandas as pd
import seaborn as sns
from neuropy.plotting import Fig
from scipy import stats
from statannotations.Annotator import Annotator

group = ["NSD", "SD"]
grpdata = subjects.GroupData()
pf_norm_tuning = grpdata.pf_norm_tuning


# replay examples
examples = grpdata.replay_examples

# replay rate
cont_events = grpdata.replay_continuous_events

fig = Fig(
    grid=(7, 6),
    hspace=0.45,
    wspace=0.5,
    constrained_layout=False,
    fontsize=5,
    axis_lw=0.8,
    tick_size=2,
)
filename = subjects.figpath_sd / "figure4"
caption = "Place cells: \n(A) Place fields recorded during NSD and SD sessions."

### replay examples

In [ ]:
subfig = fig.add_subfigure(fig.gs[1, 0:4])
axs = subfig.subplots(2, 10)

dt = 0.02 * 1000
pf_cmaps = ["Greys", "Reds"]
zts = examples.zt.unique()
for i, grp in enumerate(group):
    for i1, zt in enumerate(zts):
        events = examples[(examples.grp == grp) & (examples.zt == zt)]
        posteriors = events["posterior"].to_list()
        jumps = events["jd"].to_list()

        for i2, (p, jd) in enumerate(zip(posteriors, jumps)):
            jd = jd / p.shape[0]
            p = np.apply_along_axis(
                np.convolve, axis=0, arr=p, v=np.ones(2 * 1 + 1), mode="same"
            )
            nt = p.shape[1]

            axs_col = 2 * i1 + i2
            ax = axs[i, axs_col]

            if grp == "SD" and zt == "5-7.5":
                cmap = "Blues"
            else:
                cmap = pf_cmaps[i]

            ax.pcolormesh(p, cmap=cmap, vmin=0, vmax=0.2, rasterized=True)
            ax.text(
                0.6, 7, f"{np.round(jd,2)}", color="#039be5", fontdict=dict(fontsize=6)
            )
            ax.spines["right"].set_visible(True)
            ax.spines["top"].set_visible(True)

            if axs_col % 2 == 0 and grp == "NSD":
                ax.set_title(zt)
            if axs_col == 0:
                yticks = np.arange(p.shape[0])
                ax.set_yticks(yticks[[0, -1]], [0, 1])
            else:
                ax.set_yticks([])

            ax.set_xticks([nt // 2], [int(nt * dt)])
            ax.tick_params(length=0)

### Proportion of continuos events

In [ ]:
from statannotations.stats.StatTest import StatTest

gs = fig.subplot2grid(
    fig.gs[3:5, 0:2], grid=(2, 2), width_ratios=[2, 1], height_ratios=[2.5, 1]
)
ax = fig.subplot(gs[0])

plot_kw = dict(
    data=cont_events, x="zt", y="prop", hue="grp", hue_order=["NSD", "SD"], ax=ax
)
plotters.stripplot(**plot_kw, size=2.5)
sns.barplot(**plot_kw, facecolor="w", linewidth=0.5, edgecolor="0.2", ci=None)

# Across groups

orders = cont_events.zt.unique()
pairs = [((_, "NSD"), (_, "SD")) for _ in orders[1:]] + [
    (("0-2.5", "NSD"), ("5-7.5", "SD"))
]
# annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
# annotator.configure(test="Kruskal", **stat_kw, color="#4AB33E")
# annotator.apply_and_annotate()
# annotator.reset_configuration()

custom_long_name = "Wilcoxon_ranksum"
custom_short_name = "Wilcoxon_ranksum"
custom_func = stats.ranksums
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)

annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
annotator.configure(test="t-test_welch", **subjects.stat_kw)
# annotator.configure(
#     test=custom_test, **subjects.stat_kw, color="k", comparisons_correction=None
# )

annotator.apply_and_annotate()
annotator.reset_configuration()

# Within groups
for i, g in enumerate(["NSD", "SD"]):
    pairs2 = [((orders[_], g), (orders[_ + 1], g)) for _ in range(4)] + [
        (("PRE", g), ("0-2.5", g))
    ]
    annotator = Annotator(pairs=pairs2, **plot_kw, order=orders)
    # annotator.configure(test="Wilcoxon", **stat_kw, color=subjects.colors_sd(1)[i])
    annotator.configure(test="t-test_paired", **stat_kw, color=subjects.colors_sd(1)[i])
    annotator.apply_and_annotate()
    annotator.reset_configuration()

### ANOVA

In [ ]:
cont_events

In [ ]:
import pingouin as pg

st = pg.mixed_anova(
    data=cont_events[(cont_events.zt != "PRE")],
    dv="prop",
    within="zt",
    between="grp",
    subject="name",
)
pg.print_table(st)
posthocs = pg.pairwise_tests(
    dv="prop",
    within="zt",
    between="grp",
    subject="name",
    data=cont_events[~cont_events.zt.isin(["PRE", "MAZE"])],
)
pg.print_table(posthocs)

### saving

In [ ]:
# fig.savefig(filename)